In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

In [3]:
# Load raw data
df = pd.read_csv("Loan_Default.csv")
df.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


# Data preprocessing steps

### 1. Select features / columns

In [4]:
df.columns= df.columns.str.lower()

In [5]:
df.columns

Index(['id', 'year', 'loan_limit', 'gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'credit_worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'interest_rate_spread', 'upfront_charges', 'term', 'neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'secured_by', 'total_units',
       'income', 'credit_type', 'credit_score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'ltv', 'region', 'security_type',
       'status', 'dtir1'],
      dtype='object')

In [6]:
loan_df = df[['business_or_commercial', 'loan_amount', 'rate_of_interest', 'term', 'interest_only', 'property_value', 'income', 'credit_score', 'age','ltv', 'dtir1','status']]

In [50]:
#loan_df.to_csv("removecolumn.csv", index=False)

### 2. Remove rows with more than 3 nan values

In [9]:
def missing_data_stats(df):
    total = df.shape[0]
    columns_missing_rows = [col for col in df.columns if df[col].isnull().sum() > 0]
    missing_data_summary = {}
    for col in columns_missing_rows:
        null_count = df[col].isnull().sum()
        percent = (null_count/total) * 100
        missing_data_summary[col] = percent
        print("{} : {} ({}%)".format(col, null_count, round(percent, 2)))
    return missing_data_summary

In [10]:
missing_data_summary = missing_data_stats(loan_df)

rate_of_interest : 36439 (24.51%)
term : 41 (0.03%)
property_value : 15098 (10.16%)
income : 9150 (6.15%)
age : 200 (0.13%)
ltv : 15098 (10.16%)
dtir1 : 24121 (16.22%)


In [11]:
loan_df.shape[0]

148670

In [14]:
loan_df2 = loan_df.copy()

In [15]:
for x in range(loan_df2.shape[0]):
    loan_df2.loc[x, 'missing_value_count'] = loan_df2.iloc[x, :11].isnull().sum()

In [16]:
loan_df2.to_csv("missing_value_count.csv", index=False)

In [17]:
loan_df2 = read_csv("missing_value_count.csv")

In [51]:
loan_df2.head()

,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status,missing_value_count
0,nob/c,116500,NaN,360.0,not_int,118000.0,1740.0,758,25-34,98.728814,45.0,1,1.0
2,nob/c,406500,4.56,360.0,not_int,508000.0,9480.0,834,35-44,80.019685,46.0,0,0.0
3,nob/c,456500,4.25,360.0,not_int,658000.0,11880.0,587,45-54,69.376900,42.0,0,0.0
4,nob/c,696500,4.00,360.0,not_int,758000.0,10440.0,602,25-34,91.886544,39.0,0,0.0
5,nob/c,706500,3.99,360.0,not_int,1008000.0,10080.0,864,35-44,70.089286,40.0,0,0.0


In [20]:
limit = 3

print("Samples Before Removal : {}".format(loan_df.shape[0]))

loan_df2.drop(loan_df2[loan_df2['missing_value_count'] > limit].index, axis = 0, inplace = True)

print("Samples After Removal : {}".format(loan_df2.shape[0]))

Samples Before Removal : 148670
Samples After Removal : 133574


In [21]:
loan_df3 = loan_df2.drop(columns=["missing_value_count"])
loan_df3

,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status
0,nob/c,116500,NaN,360.0,not_int,118000.0,1740.0,758,25-34,98.728814,45.0,1
2,nob/c,406500,4.560,360.0,not_int,508000.0,9480.0,834,35-44,80.019685,46.0,0
3,nob/c,456500,4.250,360.0,not_int,658000.0,11880.0,587,45-54,69.376900,42.0,0
4,nob/c,696500,4.000,360.0,not_int,758000.0,10440.0,602,25-34,91.886544,39.0,0
5,nob/c,706500,3.990,360.0,not_int,1008000.0,10080.0,864,35-44,70.089286,40.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
148665,nob/c,436500,3.125,180.0,not_int,608000.0,7860.0,659,55-64,71.792763,48.0,0
148666,nob/c,586500,5.190,360.0,not_int,788000.0,7140.0,569,25-34,74.428934,15.0,0
148667,nob/c,446500,3.125,180.0,not_int,728000.0,6900.0,702,45-54,61.332418,49.0,0
148668,nob/c,196500,3.500,180.0,not_int,278000.0,7140.0,737,55-64,70.683453,29.0,0


In [22]:
loan_df3.drop_duplicates(inplace=True)

In [23]:
loan_df3.reset_index(drop=True, inplace=True)

In [166]:
# loan_df3.to_csv("removerows.csv", index=False)

### 3. Impute missing values

In [24]:
loan_df4 = loan_df3.copy()

In [25]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

obj_columns = [col for col in loan_df4.columns if loan_df4[col].dtype == 'object']

for col in obj_columns:
    loan_df4[col] = lb.fit_transform(loan_df4[col])

In [26]:
loan_df4

,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status
0,1,116500,NaN,360.0,1,118000.0,1740.0,758,0,98.728814,45.0,1
1,1,406500,4.560,360.0,1,508000.0,9480.0,834,1,80.019685,46.0,0
2,1,456500,4.250,360.0,1,658000.0,11880.0,587,2,69.376900,42.0,0
3,1,696500,4.000,360.0,1,758000.0,10440.0,602,0,91.886544,39.0,0
4,1,706500,3.990,360.0,1,1008000.0,10080.0,864,1,70.089286,40.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
133565,1,436500,3.125,180.0,1,608000.0,7860.0,659,3,71.792763,48.0,0
133566,1,586500,5.190,360.0,1,788000.0,7140.0,569,0,74.428934,15.0,0
133567,1,446500,3.125,180.0,1,728000.0,6900.0,702,2,61.332418,49.0,0
133568,1,196500,3.500,180.0,1,278000.0,7140.0,737,3,70.683453,29.0,0


In [27]:
loan_df4.isna().sum()

business_or_commercial        0
loan_amount                   0
rate_of_interest          21343
term                         29
interest_only                 0
property_value                2
income                     9131
credit_score                  0
age                           0
ltv                           2
dtir1                      9021
status                        0
dtype: int64

In [28]:
loan_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133570 entries, 0 to 133569
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   business_or_commercial  133570 non-null  int32  
 1   loan_amount             133570 non-null  int64  
 2   rate_of_interest        112227 non-null  float64
 3   term                    133541 non-null  float64
 4   interest_only           133570 non-null  int32  
 5   property_value          133568 non-null  float64
 6   income                  124439 non-null  float64
 7   credit_score            133570 non-null  int64  
 8   age                     133570 non-null  int32  
 9   ltv                     133568 non-null  float64
 10  dtir1                   124549 non-null  float64
 11  status                  133570 non-null  int64  
dtypes: float64(6), int32(3), int64(3)
memory usage: 10.7 MB


In [29]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors = 3)
knn.fit(loan_df4)

KNNImputer(n_neighbors=3)

In [30]:
x = knn.transform(loan_df4)

In [31]:
loan_df4.columns

Index(['business_or_commercial', 'loan_amount', 'rate_of_interest', 'term',
       'interest_only', 'property_value', 'income', 'credit_score', 'age',
       'ltv', 'dtir1', 'status'],
      dtype='object')

In [87]:
loan_df5 = pd.DataFrame(x,columns=['business_or_commercial', 'loan_amount', 'rate_of_interest', 'term',
    'interest_only', 'property_value', 'income', 'credit_score', 'age',
    'ltv', 'dtir1', 'status'])
loan_df5.head()

,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status
0,1.0,116500.0,3.583333,360.0,1.0,118000.0,1740.0,758.0,0.0,98.728814,45.0,1.0
1,1.0,406500.0,4.560000,360.0,1.0,508000.0,9480.0,834.0,1.0,80.019685,46.0,0.0
2,1.0,456500.0,4.250000,360.0,1.0,658000.0,11880.0,587.0,2.0,69.376900,42.0,0.0
3,1.0,696500.0,4.000000,360.0,1.0,758000.0,10440.0,602.0,0.0,91.886544,39.0,0.0
4,1.0,706500.0,3.990000,360.0,1.0,1008000.0,10080.0,864.0,1.0,70.089286,40.0,0.0


In [88]:
# getting id
df = pd.read_csv("Loan_Default.csv")
df_id = df['ID'].iloc[0:133570,]
df_id

0          24890
1          24891
2          24892
3          24893
4          24894
           ...  
133565    158455
133566    158456
133567    158457
133568    158458
133569    158459
Name: ID, Length: 133570, dtype: int64

In [89]:
loan_df6 = pd.concat([df_id,loan_df5],axis = 1)
loan_df6.head()

,ID,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status
0,24890,1.0,116500.0,3.583333,360.0,1.0,118000.0,1740.0,758.0,0.0,98.728814,45.0,1.0
1,24891,1.0,406500.0,4.560000,360.0,1.0,508000.0,9480.0,834.0,1.0,80.019685,46.0,0.0
2,24892,1.0,456500.0,4.250000,360.0,1.0,658000.0,11880.0,587.0,2.0,69.376900,42.0,0.0
3,24893,1.0,696500.0,4.000000,360.0,1.0,758000.0,10440.0,602.0,0.0,91.886544,39.0,0.0
4,24894,1.0,706500.0,3.990000,360.0,1.0,1008000.0,10080.0,864.0,1.0,70.089286,40.0,0.0


In [90]:
loan_df6.business_or_commercial = loan_df6.business_or_commercial.astype(int)
loan_df6.term = loan_df6.term.astype(int)
loan_df6.interest_only = loan_df6.interest_only.astype(int)
loan_df6.credit_score = loan_df6.credit_score.astype(int)
loan_df6.age = loan_df6.age.astype(int)
loan_df6.status = loan_df6.status.astype(int)

In [91]:
loan_df6.head()

,ID,business_or_commercial,loan_amount,rate_of_interest,term,interest_only,property_value,income,credit_score,age,ltv,dtir1,status
0,24890,1,116500.0,3.583333,360,1,118000.0,1740.0,758,0,98.728814,45.0,1
1,24891,1,406500.0,4.560000,360,1,508000.0,9480.0,834,1,80.019685,46.0,0
2,24892,1,456500.0,4.250000,360,1,658000.0,11880.0,587,2,69.376900,42.0,0
3,24893,1,696500.0,4.000000,360,1,758000.0,10440.0,602,0,91.886544,39.0,0
4,24894,1,706500.0,3.990000,360,1,1008000.0,10080.0,864,1,70.089286,40.0,0


In [92]:
loan_df6.isna().sum()

ID                        0
business_or_commercial    0
loan_amount               0
rate_of_interest          0
term                      0
interest_only             0
property_value            0
income                    0
credit_score              0
age                       0
ltv                       0
dtir1                     0
status                    0
dtype: int64

In [93]:
loan_df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133570 entries, 0 to 133569
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      133570 non-null  int64  
 1   business_or_commercial  133570 non-null  int32  
 2   loan_amount             133570 non-null  float64
 3   rate_of_interest        133570 non-null  float64
 4   term                    133570 non-null  int32  
 5   interest_only           133570 non-null  int32  
 6   property_value          133570 non-null  float64
 7   income                  133570 non-null  float64
 8   credit_score            133570 non-null  int32  
 9   age                     133570 non-null  int32  
 10  ltv                     133570 non-null  float64
 11  dtir1                   133570 non-null  float64
 12  status                  133570 non-null  int32  
dtypes: float64(6), int32(6), int64(1)
memory usage: 10.2 MB


In [94]:
loan_df6.to_csv("Loan_Default_knn_final.csv",index=False)